In [9]:
import sqlite3
import pandas as pd
import plotly.express as px
import nbformat
import statsmodels
import plotly.graph_objects as go

# Connect to the SQLite database
connexion = sqlite3.connect('db/vivino.db')
cursor = connexion.cursor()


In [10]:
# 1a - Marketing push - price distribution
query1a = ("""
WITH price_buckets AS (
    SELECT  NTILE(10) OVER (ORDER BY price_euros) AS price_bucket,
            price_euros
    FROM    vintages
    WHERE   bottle_volume_ml = 750
)

SELECT  price_bucket,
        AVG(price_euros) AS avg_price,
        COUNT(*) AS wines_count
FROM    price_buckets
GROUP BY price_bucket  
ORDER BY price_bucket
""")

cursor.execute(query1a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1a, connexion)
df



Number of rows: 10
[(1, 50.46808243727597, 186), (2, 91.26720430107531, 186), (3, 131.56037634408605, 186), (4, 181.5922580645162, 186), (5, 241.4904838709678, 186), (6, 328.49162162162173, 185), (7, 446.144, 185), (8, 635.586162162162, 185), (9, 964.2643243243243, 185), (10, 2686.8456216216214, 185)]


,price_bucket,avg_price,wines_count
0,1,50.468082,186
1,2,91.267204,186
2,3,131.560376,186
3,4,181.592258,186
4,5,241.490484,186
5,6,328.491622,185
6,7,446.144000,185
7,8,635.586162,185
8,9,964.264324,185
9,10,2686.845622,185


In [11]:
# 1b - Marketing push - ratings distribution
query1b = ("""
WITH ratings_avg_buckets AS (
    SELECT  NTILE(10) OVER (ORDER BY ratings_average) AS ratings_bucket,
            ratings_average
    FROM    vintages
    WHERE   bottle_volume_ml = 750  and ratings_average > 0
)

SELECT  ratings_bucket,
        MIN(ratings_average) AS avg_ratings,
        MAX(ratings_average) AS avg_ratings,
        AVG(ratings_average) AS avg_ratings,
        COUNT(*) AS wines_count
FROM    ratings_avg_buckets
GROUP BY ratings_bucket  
ORDER BY ratings_bucket
""")

cursor.execute(query1b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1b, connexion)
df


Number of rows: 10
[(1, 4.4, 4.4, 4.399999999999989, 140), (2, 4.4, 4.4, 4.399999999999989, 140), (3, 4.4, 4.5, 4.437142857142854, 140), (4, 4.5, 4.5, 4.5, 140), (5, 4.5, 4.5, 4.5, 140), (6, 4.5, 4.5, 4.5, 140), (7, 4.5, 4.6, 4.593525179856127, 139), (8, 4.6, 4.6, 4.600000000000012, 139), (9, 4.6, 4.7, 4.656115107913673, 139), (10, 4.7, 4.9, 4.722302158273366, 139)]


,ratings_bucket,avg_ratings,avg_ratings,avg_ratings,wines_count
0,1,4.4,4.4,4.400000,140
1,2,4.4,4.4,4.400000,140
2,3,4.4,4.5,4.437143,140
3,4,4.5,4.5,4.500000,140
4,5,4.5,4.5,4.500000,140
5,6,4.5,4.5,4.500000,140
6,7,4.5,4.6,4.593525,139
7,8,4.6,4.6,4.600000,139
8,9,4.6,4.7,4.656115,139
9,10,4.7,4.9,4.722302,139


In [12]:
# 1c - Marketing push - ratings count distribution
query1c = ("""
WITH ratings_count_buckets AS (
    SELECT  NTILE(10) OVER (ORDER BY ratings_count) AS ratings_count_bucket,
            ratings_count
    FROM    vintages
    WHERE   bottle_volume_ml = 750
)

SELECT  ratings_count_bucket,
        MIN(ratings_count) AS ratings_count,
        MAX(ratings_count) AS ratings_count,
        AVG(ratings_count) AS ratings_count,
        COUNT(*) AS wines_count
FROM    ratings_count_buckets
        
GROUP BY ratings_count_bucket  
ORDER BY ratings_count_bucket
""")

cursor.execute(query1c)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1c, connexion)
df


Number of rows: 10
[(1, 0, 8, 3.629032258064516, 186), (2, 8, 18, 12.688172043010752, 186), (3, 18, 39, 26.704301075268816, 186), (4, 39, 81, 56.36021505376344, 186), (5, 83, 149, 116.70430107526882, 186), (6, 149, 212, 178.71891891891892, 185), (7, 212, 309, 253.30810810810812, 185), (8, 309, 498, 393.64864864864865, 185), (9, 499, 954, 692.4918918918919, 185), (10, 959, 36681, 3299.7837837837837, 185)]


,ratings_count_bucket,ratings_count,ratings_count,ratings_count,wines_count
0,1,0,8,3.629032,186
1,2,8,18,12.688172,186
2,3,18,39,26.704301,186
3,4,39,81,56.360215,186
4,5,83,149,116.704301,186
5,6,149,212,178.718919,185
6,7,212,309,253.308108,185
7,8,309,498,393.648649,185
8,9,499,954,692.491892,185
9,10,959,36681,3299.783784,185


In [13]:
# 1d - Marketing push - final selection - add  countries
query1d = ("""
SELECT  DISTINCT
        vintages.id AS wine_id,
        vintages.name AS wine_name,
        toplists.name AS toplist_name,
        vintage_toplists_rankings.rank AS toplist_rank,
        vintages.price_euros,
        vintages.ratings_average,
        vintages.ratings_count,
        vintages.price_discount_percentage AS discount_rate
       
           
FROM    vintages
        JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
        JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
               
WHERE   
        vintages.ratings_count > 200
        AND vintages.ratings_average >= 4.6
        AND vintages.price_euros < 425
        AND vintages.bottle_volume_ml = 750
           
GROUP BY vintages.id
""")

cursor.execute(query1d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1d, connexion)
df          

Number of rows: 10
[(1485238, 'Krug Rosé', 'top_lists.EditorialTopList.2017999.name', 6, 349.0, 4.6, 3605, None), (1776981, 'Burmester Tordiz 40 Year Old Tawny Port', "Vivino's 2020 Wine Style Awards: Portuguese Tawny Port", 1, 337.5, 4.6, 474, None), (2674920, 'Ruinart Dom Ruinart Blanc de Blancs Brut Champagne 2010', "Vivino's 2016 Wine Style Awards: French Champagne", 10, 287.0, 4.6, 465, None), (5128110, 'Giacomo Conterno Barolo Cascina Francia 2012', "Vivino's 2019 Wine Style Awards: Italian Barolo", 9, 420.0, 4.6, 559, None), (12776766, 'Cerbaiona Brunello di Montalcino 2012', "Vivino's 2019 Wine Style Awards: Italian Brunello", 8, 276.25, 4.6, 380, None), (20315417, 'Achaval-Ferrer Finca Altamira Malbec 2015', "Vivino's 2020 Wine Style Awards: Argentinian Malbec", 6, 99.95, 4.6, 666, None), (25043149, 'Cardinale Red 2014', "Vivino's 2020 Wine Style Awards: Napa Valley Bordeaux Blend", 9, 399.0, 4.6, 378, None), (85559438, 'Antinori Tignanello 2016', "Vivino's 2020 Wine Style Awa

,wine_id,wine_name,toplist_name,toplist_rank,price_euros,ratings_average,ratings_count,discount_rate
0,1485238,Krug Rosé,top_lists.EditorialTopList.2017999.name,6,349.00,4.6,3605,None
1,1776981,Burmester Tordiz 40 Year Old Tawny Port,Vivino's 2020 Wine Style Awards: Portuguese Ta...,1,337.50,4.6,474,None
2,2674920,Ruinart Dom Ruinart Blanc de Blancs Brut Champ...,Vivino's 2016 Wine Style Awards: French Champagne,10,287.00,4.6,465,None
3,5128110,Giacomo Conterno Barolo Cascina Francia 2012,Vivino's 2019 Wine Style Awards: Italian Barolo,9,420.00,4.6,559,None
4,12776766,Cerbaiona Brunello di Montalcino 2012,Vivino's 2019 Wine Style Awards: Italian Brunello,8,276.25,4.6,380,None
5,20315417,Achaval-Ferrer Finca Altamira Malbec 2015,Vivino's 2020 Wine Style Awards: Argentinian M...,6,99.95,4.6,666,None
6,25043149,Cardinale Red 2014,Vivino's 2020 Wine Style Awards: Napa Valley B...,9,399.00,4.6,378,None
7,85559438,Antinori Tignanello 2016,Vivino's 2020 Wine Style Awards: Tuscan Red,10,420.00,4.6,13709,None
8,152069724,Gitana Lupi Rezerva 2017,Bestsellers in Arizona,15,36.55,4.6,1155,None
9,164942599,Krug Grande Cuvée,Vivino's 2020 Wine Style Awards: French Champa...,6,245.00,4.6,28513,None


In [14]:
# 2 - countries Marketing priorities: 

query2a= ("""
SELECT  
        countries.name AS country, 
        countries.users_count AS number_of_users,
        sum(wines.ratings_count) AS ratings_counts_tot,
        ROUND((SUM(wines.ratings_count)*1.0/countries.users_count), 2) AS ratings_counts_per_user,
        AVG(wines.ratings_average) AS ratings, 
        AVG(vintages.price_euros) AS price_EUR
        
FROM    wines
        JOIN regions ON wines.region_id = regions.id
        JOIN countries ON regions.country_code = countries.code
        JOIN vintages ON wines.id = vintages.wine_id
GROUP BY countries.name  
ORDER BY countries.users_count DESC, AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC
LIMIT 10
""") 

# for row in cursor.execute(query3):
#     print(row)
cursor.execute(query2a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query2a, connexion)
df



Number of rows: 10
[('États-Unis', 12273684, 1528232, 0.12, 4.506250000000001, 300.36616071428574), ('France', 5973301, 9634864, 1.61, 4.484049930651893, 1025.3709986130377), ('Italie', 4270717, 11898245, 2.79, 4.478492239467868, 476.0170214338511), ('Allemagne', 2549989, 16609, 0.01, 4.475000000000001, 282.01666666666665), ('Espagne', 2264396, 1132535, 0.5, 4.467175572519083, 282.1422900763359), ('Suisse', 1601799, 1047, 0.0, 4.35, 123.42500000000001), ('Portugal', 1123535, 71717, 0.06, 4.429032258064517, 161.94903225806445), ('Australie', 1022965, 142532, 0.14, 4.5, 495.00937500000003), ('Argentine', 629532, 341337, 0.54, 4.42857142857143, 111.37749999999998), ('Chili', 326757, 363988, 1.11, 4.468, 129.70519999999996)]


,country,number_of_users,ratings_counts_tot,ratings_counts_per_user,ratings,price_EUR
0,États-Unis,12273684,1528232,0.12,4.506250,300.366161
1,France,5973301,9634864,1.61,4.484050,1025.370999
2,Italie,4270717,11898245,2.79,4.478492,476.017021
3,Allemagne,2549989,16609,0.01,4.475000,282.016667
4,Espagne,2264396,1132535,0.50,4.467176,282.142290
5,Suisse,1601799,1047,0.00,4.350000,123.425000
6,Portugal,1123535,71717,0.06,4.429032,161.949032
7,Australie,1022965,142532,0.14,4.500000,495.009375
8,Argentine,629532,341337,0.54,4.428571,111.377500
9,Chili,326757,363988,1.11,4.468000,129.705200


In [15]:
# 3 - best winery >> The winery Key and name are wrong in the database, join is useless
query3a = ("""
SELECT  wines.winery_id, 
        wineries.name ,
        AVG(wines.ratings_average), 
        AVG(wines.ratings_count)        
        
FROM    wines
        JOIN wineries ON wines.winery_id = wineries.id
GROUP BY wines.winery_id  
ORDER BY AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC
LIMIT 100
""") 

# for row in cursor.execute(query3):
#     print(row)
cursor.execute(query3a)
result = cursor.fetchall()
print("Number of rows:", len(result))
#print (result)
df = pd.read_sql_query(query3a, connexion)
df


Number of rows: 4


,winery_id,name,AVG(wines.ratings_average),AVG(wines.ratings_count)
0,1651,Tenuta Tignanello 'Solaia',4.5,2472.0
1,75712,Corte di Cama Sforzato di Valtellina,4.5,58.0
2,1652,Tignanello,4.4,741.0
3,1471,Siepi,4.3,4981.0


In [16]:
# 3 - best winery 
query3b = ("""
SELECT  --wines.name AS wine_name,
        --wines.id wine_id,
        --vintages.name AS vintage_name,
        wines.winery_id,
        CASE 
                WHEN INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) > 0 THEN
                        SUBSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), 1, INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) - 1) || 
                        SUBSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) + LENGTH(wines.name))
                ELSE SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5)
        END AS winery_name,
        
        AVG(wines.ratings_average) AS ratings_avg,
        SUM(wines.ratings_count) AS ratings_cnt
        
FROM    wines
        JOIN vintages ON vintages.wine_id = wines.id
       
GROUP BY wines.winery_id  
HAVING ratings_avg > 4.5999 and ratings_cnt> 150000
ORDER BY SUM(wines.ratings_count)  DESC, AVG(wines.ratings_average) DESC
LIMIT 10
""") 

cursor.execute(query3b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query3b, connexion)
df



Number of rows: 10
[(1413, 'Tenuta San Guido ', 4.599999999999998, 2691150), (212823, 'Dom Pérignon ', 4.625000000000001, 760718), (8028, "Château d'Yquem ", 4.7, 397134), (82979, 'Masseto ', 4.700000000000002, 390816), (2107, 'Château Latour ', 4.6000000000000005, 385210), (1235, 'Louis Roederer ', 4.607692307692308, 316298), (11050, 'Vega Sicilia ', 4.666666666666667, 294013), (1301, 'Caymus ', 4.666666666666667, 240416), (1344, 'Château Angelus ', 4.6000000000000005, 206064), (12630, 'Pétrus ', 4.7, 192942)]


,winery_id,winery_name,ratings_avg,ratings_cnt
0,1413,Tenuta San Guido,4.600000,2691150
1,212823,Dom Pérignon,4.625000,760718
2,8028,Château d'Yquem,4.700000,397134
3,82979,Masseto,4.700000,390816
4,2107,Château Latour,4.600000,385210
5,1235,Louis Roederer,4.607692,316298
6,11050,Vega Sicilia,4.666667,294013
7,1301,Caymus,4.666667,240416
8,1344,Château Angelus,4.600000,206064
9,12630,Pétrus,4.700000,192942


In [17]:
# 4a - identify wine with more than 10 users at least identofued the following taste : coffee, toast, green apple, cream, and citrus
query4a = ("""

SELECT 
    --keywords.name,
    wines.id AS wine_id,
    wines.name AS wine_name,
    --keywords_wine.keyword_type AS keyword_type,
    keywords_wine.count AS keyword_count,
    COUNT(DISTINCT keywords.name) as flavor_count   
                 
FROM    keywords_wine
        JOIN keywords ON keywords.id = keywords_wine.keyword_id
        JOIN wines ON wines.id = keywords_wine.wine_id

WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') and keywords_wine.count >=10
GROUP BY wines.id
HAVING COUNT(DISTINCT keywords.name) >= 5
ORDER BY flavor_count DESC

""")

cursor.execute(query4a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query4a, connexion)
to_csv = df.to_csv('db/taste2.csv', index = False)
df

Number of rows: 19
[(7122486, 'Grande Cuvée', 713, 5), (3630450, 'Cuvée des Enchanteleurs Brut Champagne', 31, 5), (3102815, 'P2 Plénitude Brut Champagne', 161, 5), (1692390, 'Sir Winston Churchill Brut Champagne', 282, 5), (1441099, 'Blanc des Millénaires', 160, 5), (1192720, 'Dom Ruinart Blanc de Blancs Brut Champagne', 226, 5), (1167182, 'MV', 10, 5), (1164702, 'R.D Extra Brut Champagne (Récemment Dégorgé)', 163, 5), (1153863, 'Sauternes', 13, 5), (1136950, 'Comtes de Champagne Blanc de Blancs', 587, 5), (1135215, 'Sauternes (Premier Grand Cru Classé)', 10, 5), (1127349, 'Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 240, 5), (1105696, "Trebbiano d'Abruzzo", 21, 5), (86684, 'Brut Champagne', 2165, 5), (79836, 'La Grande Dame Brut Champagne', 244, 5), (79631, 'Vintage', 356, 5), (79162, 'Belle Epoque Brut Champagne', 224, 5), (74304, 'Cristal Brut Champagne (Millésimé)', 464, 5), (18931, 'La Grande Année Brut Champagne', 525, 5)]


,wine_id,wine_name,keyword_count,flavor_count
0,7122486,Grande Cuvée,713,5
1,3630450,Cuvée des Enchanteleurs Brut Champagne,31,5
2,3102815,P2 Plénitude Brut Champagne,161,5
3,1692390,Sir Winston Churchill Brut Champagne,282,5
4,1441099,Blanc des Millénaires,160,5
5,1192720,Dom Ruinart Blanc de Blancs Brut Champagne,226,5
6,1167182,MV,10,5
7,1164702,R.D Extra Brut Champagne (Récemment Dégorgé),163,5
8,1153863,Sauternes,13,5
9,1136950,Comtes de Champagne Blanc de Blancs,587,5


In [18]:
# 4b - verification of 4a
query4b = ("""

WITH taste_in_wine AS (
SELECT 
    --keywords.name,
    wines.id AS wine_id,
    wines.name AS wine_name,
    --keywords_wine.keyword_type AS keyword_type,
    keywords_wine.count AS keyword_count,
    COUNT(DISTINCT keywords.name) as flavor_count   
                 
FROM    keywords_wine
        JOIN keywords ON keywords.id = keywords_wine.keyword_id
        JOIN wines ON wines.id = keywords_wine.wine_id

WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') and keywords_wine.count >=10
GROUP BY wines.id
HAVING COUNT(DISTINCT keywords.name) >= 5
ORDER BY flavor_count DESC
)
           
SELECT DISTINCT
        taste_in_wine.wine_id,
        taste_in_wine.wine_name,
        keywords.id,
        keywords.name,
        keywords_wine.keyword_id,
        keywords_wine.count
           
FROM   taste_in_wine
        JOIN keywords_wine ON taste_in_wine.wine_id = keywords_wine.wine_id
        JOIN keywords ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') 
ORDER BY taste_in_wine.wine_id


""")

cursor.execute(query4b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query4b, connexion)
to_csv = df.to_csv('db/taste3.csv', index = False)
df

Number of rows: 95
[(18931, 'La Grande Année Brut Champagne', 417, 'toast', 417, 525), (18931, 'La Grande Année Brut Champagne', 117, 'coffee', 117, 32), (18931, 'La Grande Année Brut Champagne', 123, 'cream', 123, 425), (18931, 'La Grande Année Brut Champagne', 195, 'green apple', 195, 175), (18931, 'La Grande Année Brut Champagne', 106, 'citrus', 106, 454), (74304, 'Cristal Brut Champagne (Millésimé)', 417, 'toast', 417, 464), (74304, 'Cristal Brut Champagne (Millésimé)', 117, 'coffee', 117, 26), (74304, 'Cristal Brut Champagne (Millésimé)', 195, 'green apple', 195, 298), (74304, 'Cristal Brut Champagne (Millésimé)', 106, 'citrus', 106, 954), (74304, 'Cristal Brut Champagne (Millésimé)', 123, 'cream', 123, 496), (79162, 'Belle Epoque Brut Champagne', 195, 'green apple', 195, 224), (79162, 'Belle Epoque Brut Champagne', 417, 'toast', 417, 239), (79162, 'Belle Epoque Brut Champagne', 117, 'coffee', 117, 14), (79162, 'Belle Epoque Brut Champagne', 106, 'citrus', 106, 423), (79162, 'Bell

,wine_id,wine_name,id,name,keyword_id,count
0,18931,La Grande Année Brut Champagne,417,toast,417,525
1,18931,La Grande Année Brut Champagne,117,coffee,117,32
2,18931,La Grande Année Brut Champagne,123,cream,123,425
3,18931,La Grande Année Brut Champagne,195,green apple,195,175
4,18931,La Grande Année Brut Champagne,106,citrus,106,454
...,...,...,...,...,...,...
90,7122486,Grande Cuvée,417,toast,417,713
91,7122486,Grande Cuvée,117,coffee,117,50
92,7122486,Grande Cuvée,123,cream,123,439
93,7122486,Grande Cuvée,106,citrus,106,916


In [19]:
# 5a - Best 5 wines per type of grapes - most common grapes world wide
query5a = ("""
SELECT  DISTINCT 
        grapes.name,
        most_used_grapes_per_country.grape_id,
        most_used_grapes_per_country.wines_count AS grapes_count
                
FROM    most_used_grapes_per_country
        JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
ORDER BY grapes_count DESC
LIMIT 3

""")
cursor.execute(query5a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5a, connexion)
df


Number of rows: 3
[('Cabernet Sauvignon', 2, 801751), ('Chardonnay', 5, 604208), ('Pinot Noir', 14, 572334)]


,name,grape_id,grapes_count
0,Cabernet Sauvignon,2,801751
1,Chardonnay,5,604208
2,Pinot Noir,14,572334


In [20]:
# 5b - Best 5 wines per type of grapes -  Country list related to  most common grapes world wide
query5b = ("""

WITH most_common_grapes_WW AS (
        SELECT  DISTINCT 
                grapes.name,
                most_used_grapes_per_country.grape_id,
                most_used_grapes_per_country.wines_count AS grapes_count
               
        FROM    most_used_grapes_per_country
                JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
        ORDER BY grapes_count DESC
        LIMIT 3
)          
SELECT   
        most_used_grapes_per_country.grape_id AS grape_id,
        most_used_grapes_per_country.country_code as country_code
                   
FROM    most_used_grapes_per_country
WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
GROUP BY grape_id, country_code

           
 

""")
cursor.execute(query5b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5b, connexion)
df

Number of rows: 21
[(2, 'ar'), (2, 'au'), (2, 'cl'), (2, 'es'), (2, 'gr'), (2, 'hr'), (2, 'hu'), (2, 'il'), (2, 'md'), (2, 'ro'), (2, 'us'), (2, 'za'), (5, 'ar'), (5, 'au'), (5, 'fr'), (5, 'it'), (5, 'md'), (5, 'us'), (14, 'ch'), (14, 'fr'), (14, 'us')]


,grape_id,country_code
0,2,ar
1,2,au
2,2,cl
3,2,es
4,2,gr
5,2,hr
6,2,hu
7,2,il
8,2,md
9,2,ro


In [21]:
# 5c - Best 5 wines per type of grapes -  region list  for countries related to most common grapes WW
query5c = ("""

WITH countries_most_common_grapes AS (
    WITH most_common_grapes_WW AS (
            SELECT  DISTINCT 
                    grapes.name,
                    most_used_grapes_per_country.grape_id,
                    most_used_grapes_per_country.wines_count AS grapes_count
                
            FROM    most_used_grapes_per_country
                    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            ORDER BY grapes_count DESC
            LIMIT 3
    )          
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
    GROUP BY grape_id, country_code
)
           
SELECT
    regions.name AS region_name,
    regions.id AS region_id,
    regions.country_code AS region_country_code,
    countries_most_common_grapes.country_code AS prev_country_code,
    countries_most_common_grapes.grape_id AS grape_id
    
    
FROM    
    regions
    JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
    
ORDER BY grape_id, region_id, region_country_code

           
        
""")


cursor.execute(query5c)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5c, connexion)
df

Number of rows: 410
[('South Australia', 7, 'au', 'au', 2), ('California', 24, 'us', 'us', 2), ('Napa Valley', 25, 'us', 'us', 2), ('Sonoma County', 26, 'us', 'us', 2), ('Oakville', 42, 'us', 'us', 2), ('Willamette Valley', 51, 'us', 'us', 2), ('Yamhill County', 55, 'us', 'us', 2), ('Lodi', 65, 'us', 'us', 2), ('Howell Mountain', 85, 'us', 'us', 2), ('Santa Cruz Mountains', 86, 'us', 'us', 2), ('Stags Leap District', 88, 'us', 'us', 2), ('Calistoga', 91, 'us', 'us', 2), ('Russian River Valley', 94, 'us', 'us', 2), ('Los Carneros', 96, 'us', 'us', 2), ('Sta. Rita Hills', 99, 'us', 'us', 2), ('Alexander Valley', 100, 'us', 'us', 2), ('Rutherford', 105, 'us', 'us', 2), ('Walla Walla Valley', 324, 'us', 'us', 2), ('Columbia Valley', 327, 'us', 'us', 2), ('Barossa Valley', 397, 'au', 'au', 2), ('Ribera del Duero', 405, 'es', 'es', 2), ('Cachapoal Valley', 411, 'cl', 'cl', 2), ('Margaret River', 425, 'au', 'au', 2), ('Toro', 430, 'es', 'es', 2), ('Clare Valley', 444, 'au', 'au', 2), ('Mendoz

,region_name,region_id,region_country_code,prev_country_code,grape_id
0,South Australia,7,au,au,2
1,California,24,us,us,2
2,Napa Valley,25,us,us,2
3,Sonoma County,26,us,us,2
4,Oakville,42,us,us,2
...,...,...,...,...,...
405,Champagne Grand Cru 'Ambonnay',4624,fr,fr,14
406,Champagne Grand Cru 'Avize',4625,fr,fr,14
407,Champagne Grand Cru 'Le Mesnil-sur-Oger',4631,fr,fr,14
408,Champagne Grand Cru,4638,fr,fr,14


In [62]:
# 5d - Best 5 wines per type of grapes - all together and extracting the 5 best for each grapes from all region in  the region list
query5d = ("""

WITH countries_most_common_grapes AS (
    WITH most_common_grapes_WW AS (
            SELECT  DISTINCT 
                    grapes.name,
                    most_used_grapes_per_country.grape_id,
                    most_used_grapes_per_country.wines_count AS grapes_count
                
            FROM    most_used_grapes_per_country
                    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            ORDER BY grapes_count DESC
            LIMIT 3
    )          
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
    GROUP BY grape_id, country_code
),

ranked_wines AS ( 
        SELECT                  
                ROW_NUMBER() OVER(
                        PARTITION BY countries_most_common_grapes.grape_id 
                        ORDER BY vintages.ratings_average DESC, 
                        vintages.ratings_count DESC
                        ) as row_nb,
                grapes.name AS grape_name,
                vintages.name as vintage_name,
                vintages.ratings_average AS vintages_ratings,
                vintages.ratings_count as vintages_ratings_count,
                regions.name AS region_name,
                regions.country_code AS region_country_code
                              
                   
                
        FROM    regions
                JOIN vintages ON wines.id = vintages.wine_id
                JOIN wines ON regions.id = wines.region_id
                JOIN countries  ON regions.country_code = countries.code
                --JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
                JOIN countries_most_common_grapes ON countries.code = countries_most_common_grapes.country_code
                JOIN grapes ON countries_most_common_grapes.grape_id = grapes.id
                
)
           
SELECT * FROM ranked_wines WHERE row_nb <= 5
        
""")


cursor.execute(query5d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5d, connexion)
df

Number of rows: 15
[(1, 'Cabernet Sauvignon', 'Scarecrow Cabernet Sauvignon 2015', 4.8, 243, 'Rutherford', 'us'), (2, 'Cabernet Sauvignon', 'Colgin IX Estate Red 2016', 4.8, 131, 'Napa Valley', 'us'), (3, 'Cabernet Sauvignon', 'Harlan Estate Harlan Estate Red 1998', 4.8, 104, 'Napa Valley', 'us'), (4, 'Cabernet Sauvignon', 'Artadi Viña El Pison 2018', 4.8, 64, 'Vino de España', 'es'), (5, 'Cabernet Sauvignon', 'Opus One Opus One 2015', 4.7, 4948, 'Napa Valley', 'us'), (1, 'Chardonnay', 'Biondi-Santi Brunello di Montalcino Riserva 1955', 4.9, 35, 'Brunello di Montalcino', 'it'), (2, 'Chardonnay', "Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001", 4.9, 32, 'Sauternes', 'fr'), (3, 'Chardonnay', 'Pétrus Pomerol 1990', 4.8, 1190, 'Pomerol', 'fr'), (4, 'Chardonnay', 'Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne 1996', 4.8, 1004, "Champagne Grand Cru 'Le Mesnil-sur-Oger'", 'fr'), (5, 'Chardonnay', 'Château Lafite Rothschild Pauillac (Premier Grand Cru Classé) 2003'

,row_nb,grape_name,vintage_name,vintages_ratings,vintages_ratings_count,region_name,region_country_code
0,1,Cabernet Sauvignon,Scarecrow Cabernet Sauvignon 2015,4.8,243,Rutherford,us
1,2,Cabernet Sauvignon,Colgin IX Estate Red 2016,4.8,131,Napa Valley,us
2,3,Cabernet Sauvignon,Harlan Estate Harlan Estate Red 1998,4.8,104,Napa Valley,us
3,4,Cabernet Sauvignon,Artadi Viña El Pison 2018,4.8,64,Vino de España,es
4,5,Cabernet Sauvignon,Opus One Opus One 2015,4.7,4948,Napa Valley,us
5,1,Chardonnay,Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,35,Brunello di Montalcino,it
6,2,Chardonnay,Château Doisy-Daëne L'Extravagant de Doisy-Daë...,4.9,32,Sauternes,fr
7,3,Chardonnay,Pétrus Pomerol 1990,4.8,1190,Pomerol,fr
8,4,Chardonnay,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...,4.8,1004,Champagne Grand Cru 'Le Mesnil-sur-Oger',fr
9,5,Chardonnay,Château Lafite Rothschild Pauillac (Premier Gr...,4.8,1000,Pauillac,fr


In [54]:
# 5e - calculate the ration of the grape name in the  wine name

query5e = ("""
SELECT 
        grapes.name AS grape_name,
        COUNT(wines.name) AS wine_name_count,
        COUNT(wines.name)*100.0/(SELECT(COUNT(wines.name)) from wines) AS "ratio_(%)"

FROM  grapes
      JOIN wines ON wines.name LIKE '%' || grapes.name || '%'
        
WHERE grapes.name IN ('Chardonnay', 'Cabernet Sauvignon', 'Pinot Noir')
           
GROUP BY grapes.name
           
UNION ALL 

SELECT 
           'Total_nb_wines' AS grape_name,
            COUNT(wines.name) AS wine_name_count,
            100.0 as "ratio_(%)"
FROM wines

""")

cursor.execute(query5e)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5e, connexion)
df

Number of rows: 4
[('Cabernet Sauvignon', 29, 2.843137254901961), ('Chardonnay', 8, 0.7843137254901961), ('Pinot Noir', 10, 0.9803921568627451), ('Total_nb_wines', 1020, 100.0)]


,grape_name,wine_name_count,ratio_(%)
0,Cabernet Sauvignon,29,2.843137
1,Chardonnay,8,0.784314
2,Pinot Noir,10,0.980392
3,Total_nb_wines,1020,100.000000


In [1]:
# 6a - Country rates on wines vizualisation 
query6a= ("""
SELECT  countries.name AS Country,
        countries.users_count AS users_nb,
        AVG(wines.ratings_average) AS ratings_avg, 
        SUM(wines.ratings_count) AS 'total_ratings',
        ROUND(SUM(wines.ratings_count)*1.0/countries.users_count,2) AS 'ratings/user',
        countries.wines_count AS wines_count,
        countries.wineries_count AS wineries_nb
        
        
        
        
FROM    countries
        JOIN regions ON countries.code = regions.country_code
        JOIN wines ON regions.id = wines.region_id

GROUP BY countries.name
ORDER BY AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC

""") 

cursor.execute(query6a)
result = cursor.fetchall()
print("Number of rows:", len(result))
#print (result)
df = pd.read_sql_query(query6a, connexion)
print(df)




fig1 = px.bar(df.sort_values('ratings_avg', ascending=False), 
             x='Country', 
             y='ratings_avg', 
             title='Wines Rating per Country',
             labels={'ratings_avg': '', 'Country': ''},
             color='ratings_avg',
             color_continuous_scale=px.colors.sequential.Magenta)

# Improve the layout
fig1.update_yaxes(range=[4, max(df['ratings_avg'])], dtick=0.5)
fig1.update_layout(
    title={
        'text': "<b>Wines Rating per country of production</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 26}
        })

fig1.add_annotation(
    xref='paper',
    yref='paper',
    x=-0.04,
    y=1.15,
    text='<b>Rating</b>',
    showarrow=False,
    font=dict(
        size=14,
        color="black"
    ),
    align='left',
    ax=0,
    ay=0,
    bordercolor='white',
    borderwidth=1,
    borderpad=4
)

fig1.show()



NameError: name 'cursor' is not defined

In [60]:
#7 - Cabernet sauvignon recommendation

query5d = ("""

WITH countries_most_common_grapes AS (
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id = 2 
    
),

ranked_vintage AS ( 
        SELECT
                --countries_most_common_grapes.grape_id AS grape_id,
                grapes.name AS grape_name,
                vintages.name as vintage_name,
                vintages.ratings_average AS vintages_ratings,
                vintages.ratings_count as vintages_ratings_count,
                vintages.price_euros AS price,
                --wines.name as wines_name,
                --wines.ratings_average AS ratings,
                --wines.ratings_count as ratings_count,
                regions.name AS region_name,
                regions.country_code AS region_country_code
                --countries_most_common_grapes.country_code AS prev_country_code
                --wines.region_id as wines_region_id,
                --regions.id AS region_id,
                
                   
                
        FROM    regions
                JOIN vintages ON wines.id = vintages.wine_id
                JOIN wines ON regions.id = wines.region_id
                --JOIN countries  ON regions.country_code = countries.code
                JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
                JOIN grapes ON countries_most_common_grapes.grape_id = grapes.id
)
           
SELECT * FROM ranked_vintage 
ORDER BY price DESC
LIMIT 20
           
        
""")


cursor.execute(query5d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5d, connexion)
df

Number of rows: 20
[('Cabernet Sauvignon', 'Dominio de Pingus Pingus 2004', 4.7, 273, 3395.0, 'Ribera del Duero', 'es'), ('Cabernet Sauvignon', 'Dominio de Pingus Pingus 2014', 4.6, 174, 2668.75, 'Ribera del Duero', 'es'), ('Cabernet Sauvignon', 'Dominio de Pingus Pingus 2005', 4.7, 290, 2327.5, 'Ribera del Duero', 'es'), ('Cabernet Sauvignon', 'Descendientes de J. Palacios La Faraona Bierzo (Corullón) 2014', 0.0, 20, 2327.5, 'Bierzo', 'es'), ('Cabernet Sauvignon', 'Descendientes de J. Palacios La Faraona Bierzo (Corullón) 2016', 0.0, 24, 2071.25, 'Bierzo', 'es'), ('Cabernet Sauvignon', 'Dominio de Pingus Pingus 2015', 4.7, 223, 2028.75, 'Ribera del Duero', 'es'), ('Cabernet Sauvignon', 'Harlan Estate Harlan Estate Red 1998', 4.8, 104, 1921.25, 'Napa Valley', 'us'), ('Cabernet Sauvignon', 'Dominio de Pingus Pingus 2013', 4.6, 330, 1665.0, 'Ribera del Duero', 'es'), ('Cabernet Sauvignon', 'Opus One Opus One 2015', 4.7, 4948, 1665.0, 'Napa Valley', 'us'), ('Cabernet Sauvignon', 'Scarecro

,grape_name,vintage_name,vintages_ratings,vintages_ratings_count,price,region_name,region_country_code
0,Cabernet Sauvignon,Dominio de Pingus Pingus 2004,4.7,273,3395.00,Ribera del Duero,es
1,Cabernet Sauvignon,Dominio de Pingus Pingus 2014,4.6,174,2668.75,Ribera del Duero,es
2,Cabernet Sauvignon,Dominio de Pingus Pingus 2005,4.7,290,2327.50,Ribera del Duero,es
3,Cabernet Sauvignon,Descendientes de J. Palacios La Faraona Bierzo...,0.0,20,2327.50,Bierzo,es
4,Cabernet Sauvignon,Descendientes de J. Palacios La Faraona Bierzo...,0.0,24,2071.25,Bierzo,es
5,Cabernet Sauvignon,Dominio de Pingus Pingus 2015,4.7,223,2028.75,Ribera del Duero,es
6,Cabernet Sauvignon,Harlan Estate Harlan Estate Red 1998,4.8,104,1921.25,Napa Valley,us
7,Cabernet Sauvignon,Dominio de Pingus Pingus 2013,4.6,330,1665.00,Ribera del Duero,es
8,Cabernet Sauvignon,Opus One Opus One 2015,4.7,4948,1665.00,Napa Valley,us
9,Cabernet Sauvignon,Scarecrow Cabernet Sauvignon 2015,4.8,243,1558.75,Rutherford,us
